## Install Libraries

In [1]:
!pip install -qq arize-phoenix llama-index "openai>=1" gcsfs nest_asyncio langchain langchain-community cohere llama-index-postprocessor-cohere-rerank

## Set Up Environment Variables


In [2]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

if not (cohere_api_key := os.getenv("COHERE_API_KEY")):
    cohere_api_key = getpass("🔑 Enter your Cohere API key: ")
os.environ["COHERE_API_KEY"] = cohere_api_key

🔑 Enter your OpenAI API key: ··········
🔑 Enter your Cohere API key: ··········


## Launch Phoenix and Setup instrumentation

In [3]:
import phoenix as px

In [4]:
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://9zpo69ffpri12-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [5]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

## Parse Pheonix Documentation into Llama-index Documents

In [6]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import json
import logging
import sys
import nest_asyncio
import time

nest_asyncio.apply()

import pandas as pd
from langchain.docstore.document import Document as LangChainDocument
from langchain.document_loaders import GitbookLoader
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

WARNI [langchain_community.utils.user_agent] USER_AGENT environment variable not set, consider setting it to identify your requests.


Enable Phoenix tracing via `LlamaIndexInstrumentor`. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [7]:
"""
Fetches the Arize documentation from Gitbook and serializes it into LangChain format.
"""


def load_gitbook_docs(docs_url: str):
    """Loads documents from a Gitbook URL.

    Args:
        docs_url (str): URL to Gitbook docs.

    Returns:
        List[LangChainDocument]: List of documents in LangChain format.
    """
    loader = GitbookLoader(
        docs_url,
        load_all_paths=True,
    )
    return loader.load()


logging.basicConfig(level=logging.INFO, stream=sys.stdout)

    # fetch documentation
docs_url = "https://docs.arize.com/phoenix"
embedding_model_name = "text-embedding-ada-002"
docs = load_gitbook_docs(docs_url)


/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Fetching pages: 100%|##########| 126/126 [00:57<00:00,  2.18it/s]


In [8]:
documents = []
for doc in docs:
  documents.append(Document(metadata=doc.metadata, text=doc.page_content))

In [9]:
documents[0].metadata

{'source': 'https://docs.arize.com/phoenix/', 'title': 'Arize Phoenix'}

In [10]:
import json

# Convert documents to a JSON serializable format (if needed)
documents_json = [doc.to_dict() for doc in documents]

# Save to a JSON file
with open('llama_index_documents.json', 'w') as file:
    json.dump(documents_json, file, indent=4)


## Setup VectorStore and Query Engine

In [11]:
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.postprocessor.cohere_rerank import CohereRerank

# Define an LLM
llm = OpenAI(model="gpt-4")

# Build index with a chunk_size of 1024
# node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
# nodes = node_parser.get_nodes_from_documents(documents)
splitter = SentenceSplitter(chunk_size=1024,chunk_overlap=250)
nodes = splitter.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

Build a QueryEngine and start querying.

In [12]:
cohere_api_key = os.environ["COHERE_API_KEY"]
cohere_rerank = CohereRerank(api_key=cohere_api_key, top_n=2)

query_engine = vector_index.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[cohere_rerank],
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


## Import Questions

In [13]:
questions_df = pd.read_parquet("/content/PhoenixRAGUseCaseQuestions.parquet")

In [ ]:
questions_df

,Prompt/ Question
0,How do I send traces to Phoenix?
1,What happens if I send the same traces twice?
2,Which frameworks and LLM providers are support...
3,How can users create and manage datasets for p...
4,How does Arize Phoenix use OpenTelemetry?
...,...
95,What is the Data retention policy?
96,Will hosted Phoenix be on the latest version o...
97,Is Hosted Phoenix free?
98,Can I persist data in the notebook?


## Generate Answers for all of the questions

In [14]:
# loop over the questions and generate the answers
for i, row in questions_df.iterrows():
    if i in [25, 50, 75]:
        time.sleep(30)
    question = row["Prompt/ Question"]
    response_vector = query_engine.query(question)
    print(f"Question: {question}\nAnswer: {response_vector.response}\n")

Question: How do I send traces to Phoenix?
Answer: You can send traces to Phoenix by instrumenting your application either manually or automatically. Once the application is instrumented, the spans created via instrumentation are exported to a collector using an exporter. The Phoenix server acts as the collector and receives the spans from any application(s) that is exporting spans to it. The OpenTelemetry Protocol (OTLP) is used to transmit the traces from your application to the Phoenix collector, with Phoenix currently supporting OTLP over HTTP.

Question: What happens if I send the same traces twice? 
Answer: Tracing records the paths taken by requests as they move through multiple steps. Sending the same traces twice would likely result in duplicate records of the paths taken by those requests, potentially leading to redundant information being captured in the tracing system.

Question: Which frameworks and LLM providers are supported by Arize Phoenix for seamless integration?
Ans

## Pheonix Evals

In [25]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
9f030a063826726e 0                  c7592c02a4f76318adde85b4febe2f62   
                 1                  c7592c02a4f76318adde85b4febe2f62   
                 2                  c7592c02a4f76318adde85b4febe2f62   
                 3                  c7592c02a4f76318adde85b4febe2f62   
                 4                  c7592c02a4f76318adde85b4febe2f62   
...                                                              ...   
fc02f9e9d95f7f85 0                  9af3cfbb6cc446d32032feb06ca32a45   
                 1                  9af3cfbb6cc446d32032feb06ca32a45   
                 2                  9af3cfbb6cc446d32032feb06ca32a45   
                 3                  9af3cfbb6cc446d32032feb06ca32a45   
                 4                  9af3cfbb6cc446d32032feb06ca32a45   

                                                                   input  \
context.span_id  document_position                                         
9f030a063826726e 0                      How do I send traces to Phoenix?   
                 1                      How do I send traces to Phoenix?   
                 2                      How do I send traces to Phoenix?   
                 3                      How do I send traces to Phoenix?   
                 4                      How do I send traces to Phoenix?   
...                                                                  ...   
fc02f9e9d95f7f85 0                  Can I use gRPC for trace collection?   
                 1                  Can I use gRPC for trace collection?   
                 2                  Can I use gRPC for trace collection?   
                 3                  Can I use gRPC for trace collection?   
                 4                  Can I use gRPC for trace collection?   

                                                                            reference  \
context.span_id  document_position                                                      
9f030a063826726e 0                  Tracing Core Concepts\nHow to log traces\nTo l...   
                 1                  How does Tracing Work?\nThe components behind ...   
                 2                  Quickstart: Tracing\nInspect the inner-working...   
                 3                  Save and Load Traces\nHow to manually save and...   
                 4                  Trace a Deployed App\nOnce you are done iterat...   
...                                                                               ...   
fc02f9e9d95f7f85 0                  Use Cases: Tracing\nThe following. guides serv...   
                 1                  Overview: Tracing\nTracing the execution of LL...   
                 2                  How does Tracing Work?\nThe components behind ...   
                 3                  os\n.\nenviron\n[\n"PHOENIX_NOTEBOOK_ENV"\n]\n...   
                 4                  tracer_provider \n=\n trace_sdk\n.\nTracerProv...   

                                    document_score  
context.span_id  document_position                  
9f030a063826726e 0                        0.841724  
                 1                        0.840770  
                 2                        0.831993  
                 3                        0.821851  
                 4                        0.821648  
...                                            ...  
fc02f9e9d95f7f85 0                        0.773752  
                 1                        0.771979  
                 2                        0.767665  
                 3                        0.767380  
                 4                        0.765412  

[500 rows x 4 columns]

In [26]:
from phoenix.session.evaluation import get_qa_with_reference

queries_df = get_qa_with_reference(px.active_session())
queries_df

,input,output,reference
context.span_id,,,
54cdf11c954ecc3f,How do I send traces to Phoenix?,You can send traces to Phoenix by instrumentin...,Tracing Core Concepts\nHow to log traces\nTo l...
5c89706b00c67ba9,What happens if I send the same traces twice?,Tracing records the paths taken by requests as...,The reply half may be formatted for response p...
3ef54d0c9aee79ee,Which frameworks and LLM providers are support...,Arize Phoenix supports a variety of frameworks...,"Arize\nPhoenix works hand-in-hand with Arize, ..."
9c0f677d46d73b9d,How can users create and manage datasets for p...,Users can create and manage datasets for promp...,"Arize\nPhoenix works hand-in-hand with Arize, ..."
b7b06e01d71b7fef,How does Arize Phoenix use OpenTelemetry?,Arize Phoenix uses OpenTelemetry for instrumen...,Arize Phoenix\nAI Observability and Evaluation...
...,...,...,...
aa5c48651130d333,What is the Data retention policy?,The data retention policy allows users to pers...,Email Extraction\nComing soon\nPrevious\nSumma...
789bd79b491d47c9,Will hosted Phoenix be on the latest version o...,Hosted Phoenix will always be on the latest ve...,Deployment\nHow to self-host a phoenix instanc...
630103d91a7d81e1,Is Hosted Phoenix free?,"Hosted Phoenix is free for all developers, wit...",Hosted Phoenix\nWe now offer a hosted version ...


Let's now use Phoenix's LLM Evals to evaluate the relevance of the retrieved documents with regards to the query. Note, we've turned on `explanations` which prompts the LLM to explain it's reasoning. This can be useful for debugging and for figuring out potential corrective actions.

In [27]:
from phoenix.evals import (
    OpenAIModel,
    RelevanceEvaluator,
    HallucinationEvaluator,
    QAEvaluator,
    run_evals,
)

eval_model = OpenAIModel(model="gpt-4")
relevance_evaluator = RelevanceEvaluator(eval_model)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_evaluator = QAEvaluator(eval_model)

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

run_evals |          | 0/500 (0.0%) | ⏳ 00:00<? | ?it/s

In [28]:
hallucination_eval_df, qa_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_evaluator],
    provide_explanation=True,
    concurrency=20,

)

run_evals |          | 0/200 (0.0%) | ⏳ 00:00<? | ?it/s

In [29]:
retrieved_documents_relevance_df = retrieved_documents_relevance_df.reset_index().set_index('context.span_id')
retrieved_documents_relevance_df

,document_position,label,score,explanation
context.span_id,,,,
9f030a063826726e,0,relevant,1,The question asks about how to send traces to ...
9f030a063826726e,1,relevant,1,The question asks about how to send traces to ...
9f030a063826726e,2,relevant,1,The question asks about how to send traces to ...
9f030a063826726e,3,unrelated,0,The question asks about how to send traces to ...
9f030a063826726e,4,relevant,1,The question asks about how to send traces to ...
...,...,...,...,...
fc02f9e9d95f7f85,0,unrelated,0,The question asks about the possibility of usi...
fc02f9e9d95f7f85,1,unrelated,0,The question asks about the possibility of usi...
fc02f9e9d95f7f85,2,unrelated,0,The question is asking about the possibility o...


In [30]:
hallucination_eval_df.head()

,label,score,explanation
context.span_id,,,
54cdf11c954ecc3f,factual,0.0,The query asks about how to send traces to Pho...
5c89706b00c67ba9,hallucinated,1.0,The query asks what happens if the same traces...
3ef54d0c9aee79ee,factual,0.0,The query asks about the frameworks and LLM pr...
9c0f677d46d73b9d,factual,0.0,The reference text provides information on how...
b7b06e01d71b7fef,factual,0.0,The query asks about how Arize Phoenix uses Op...


## Log the Evals into Phoenix

In [31]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_eval_df),
    SpanEvaluations(eval_name="Retrieval Relevance", dataframe=retrieved_documents_relevance_df)
)

In [16]:
session.view()

📺 Opening a view to the Phoenix app. The app is running at https://9zpo69ffpri13-496ff2e9c6d22116-6006-colab.googleusercontent.com/


## Save the Trace

In [ ]:
import os

# Specify and Create the Directory for Trace Dataset
directory = 'my_saved_traces'
os.makedirs(directory, exist_ok=True)

# Save the Trace Dataset
trace_id = px.Client().get_trace_dataset().save(directory=directory)